In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/brady/git/MolecularNodes/docs':
  os.chdir(r'/Users/brady/git/MolecularNodes/docs')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


In [2]:
#| label: setup
#| code-fold: true
import molecularnodes as mn
import bpy
import sys
import tempfile
import os
from IPython.display import display, Image

sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')
mn.register()

def clear_scene():
    bpy.ops.object.select_all(action="DESELECT")
    bpy.ops.object.select_by_type(type="MESH")
    bpy.ops.object.delete()
    for node in bpy.data.node_groups:
        if node.type == "GEOMETRY":
            bpy.data.node_groups.remove(node)

def orient_camera(object):
    object.select_set(True)
    bpy.ops.view3d.camera_to_view_selected()

def render_image(engine = 'eevee', x = 1000, y = 500):
    # setup render engine
    if engine == "eevee":
        bpy.context.scene.render.engine = "BLENDER_EEVEE"
    elif engine == "cycles":
        
        bpy.context.scene.render.engine = "CYCLES"
        try:
            bpy.context.scene.cycles.device = "GPU"
        except:
            print("GPU Rendering not available")

    # Render
    with tempfile.TemporaryDirectory() as temp:

        path = os.path.join(temp, "test.png")
        bpy.context.scene.render.resolution_x = x
        bpy.context.scene.render.resolution_y = y
        bpy.context.scene.render.image_settings.file_format = "PNG"
        bpy.context.scene.render.filepath = path
        bpy.ops.render.render(write_still=True)
        display(Image(filename=path))

In [3]:
#| label: setup-scene
#| code-fold: true

# load template scene with nice HDRI lighting
bpy.ops.wm.read_homefile(app_template = "MolecularNodes")

# change the background to a custom color
try:
    world_nodes = bpy.data.worlds['World Shader'].node_tree.nodes
    world_nodes['MN_world_shader'].inputs['BG Color'].default_value = mn.color.random_rgb()
except KeyError:
    print("Oh no, didn't set up the base scene.")

In [4]:
clear_scene()
mol = mn.load.molecule_rcsb('4ozs', build_assembly=True, starting_style="cartoon")
mol.rotation_euler = (0, 3.14 / 2, 0)
mol.select_set(True)
bpy.ops.view3d.camera_to_view_selected()
render_image()

In [5]:
clear_scene()
mol = mn.load.molecule_rcsb('8HXZ', center_molecule = True, starting_style="preset_3")
orient_camera(mol)
render_image()

In [6]:
render_image('cycles')

In [7]:
clear_scene()

mol = mn.load.molecule_rcsb('6N2Y', center_molecule=True, starting_style="preset_3")
mol.select_set(True)
mol.rotation_euler = (0, 90, 180)
orient_camera(mol)
render_image()

In [8]:
clear_scene()
mol = mn.load.molecule_rcsb('8CPY', build_assembly=True, starting_style="surface")
orient_camera(mol)
render_image('cycles')